# TODO: Review commented out parameters in lvae TopDownLayer and reimplement
1) TODO when we meet in person: Review if I implemented head_z_dims correctly (done)
2) Unknown params:
- use_mode (never used?) # ok to remove. -> depends on mode_layers # remove also this (it's for sampling that we don't need)
- force_constant_output (never used?) # dunno (removed)
- difference between use_uncond_mode and inference_mode?
    - in TopDownLayer.forward the merge layer is commented out, so use_uncond_mode is uneffective. Why? Does it matter?
        - TODO: YES. This is one of the parts to ablate. It is currently disabled because it lead to better results, we need to make it switchable for-layer.
          - there is self.use_uncond_mode_at that can be used for this.
        - This has been renamed to use_skip_connections (done)
3) There are A LOT of places when the networks are on the wrong device.
    - Should be fixed (done)
4) Do we need to ablate bu_values or skip_connections in TopDownLayer?
    - both
5) Check if the KL handling in inference is correct (MixtureStochasticConvBlock.forward when label is None) (done)

- Entropy should be removed (done)
- "pi" became "class_probabilities" (done)


-  !!!! Threshold: Should be added as a scheduler and decrease donly in semisupervised mode (see trainer.py line 284 in Sheida's repo)

- 


## TODOS
- Log validation losses too
- [ ] check data_mean and data_std are actually needed in this model (disabled for now)
- [ ] add .get_data_statistics() to the datamodule
- [ ] Reimplement data_mean and data_std handling where appropriate
- Mask and Label size depend on the "wait_count" (i.e., patience) of EarlyStopping in the original training script. Is that correct?
- Also ModeSwitchCallback depends on EarlyStopping patience.
- I noticed that it actually resets the wait_count to 0, need to find a way to do it.
- ...shouldn't we just do it when the EarlyStopping triggers?
- Implement WandbLogger callback and eventually tensorboard logging too
- Implement dataset
- Train :)

In [ ]:
from eps_seg.config import LVAEConfig
from eps_seg.models import LVAEModel
import torch 

config = LVAEConfig()
print(config)

x = torch.randn(2, 1, 64, 64)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = x.to(device)
model = LVAEModel(config).to(device)
model.eval()
model(x) # This DOES NOT normalize data!

In [ ]:
import pytorch_lightning as pl

from eps_seg.config.train import TrainConfig
from eps_seg.train.callbacks import ModelCheckpoint, WandbLogger, EarlyStoppingWithWaitCount, ThresholdSchedulerCallback, MaskLabelSizeSchedulerCallback, ModeSwitchCallback, RadiusIncreaseCallback

train_config = TrainConfig()
print(train_config)

model_name=(FieldInfo(annotation=NoneType, required=False, default='eps_seg_default', description='Name of the model'),) lr=3e-05 max_epochs=1000 batch_size=256 amp=True gradient_scale=256 max_grad_norm=1.0 alpha=1.0 beta=0.1 gamma=1.0 use_wandb=True initial_label_size=1 final_label_size=10 initial_mask_size=1 final_mask_size=10 step_interval=20


In [ ]:
callbacks = []

# Fixme: Use lightning built-ins
callbacks.append(ModelCheckpoint(monitor="val_total", save_top_k=1))
callbacks.append(EarlyStoppingWithWaitCount(monitor="val_total", patience=50))
# FIXME: Use correct model checkpoint folder
callbacks.append(ModeSwitchCallback(dirpath=model.train_cfg.model_name,
                                    patience_threshold=50))
callbacks.append(RadiusIncreaseCallback(dirpath=model.train_cfg.model_name,
                                       patience_threshold=50))


# TODO: Write a Callback not a Scheduler
if train_config.initial_label_size != train_config.final_label_size:
    callbacks.append(
        MaskLabelSizeSchedulerCallback(
            initial_label=train_config.initial_label_size,
            final_label=train_config.final_label_size,
            initial_mask=train_config.initial_mask_size,
            final_mask=train_config.final_mask_size,
            step_interval=train_config.step_interval,
            use_patience=True,
        )
    )

# TODO: Mode Switch Callback before this

callbacks.append(ThresholdSchedulerCallback(
    initial_threshold=train_config.initial_threshold,
    max_threshold=train_config.max_threshold,
    step_size=train_config.threshold_step,
    mode="semisupervised"
))



trainer = pl.Trainer(
    devices=1, 
    max_epochs=train_config.max_epochs,
    logger=WandbLogger(project=train_config.model_name) if train_config.use_wandb else None,
    callbacks=callbacks,
    precision = "16-mixed" if train_config.amp else 32,
    gradient_clip_val=train_config.max_grad_norm, 
    log_every_n_steps=train_config.log_every_n_steps,
    )

In [4]:
from eps_seg.train.optimizers import LabelSizeScheduler
label_size_scheduler = LabelSizeScheduler(
    initial_size=train_config.initial_label_size,
    final_size=train_config.final_label_size,
    step_interval=train_config.step_interval,
)

In [12]:
for s in range(100):
    print(label_size_scheduler.get_label_size(current_step=s))

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
